In [1]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)

In [2]:
# data directory
data_dir = 'data/'

In [3]:
from my_module import train_and_evaluate
from my_module import load_datasets_sec

In [5]:
lgbm2_features = ['stock_id', 'time_id', 'target', 'row_id', 'wap1_sum', 'wap1_mean',
       'wap1_std', 'wap2_sum', 'wap2_mean', 'wap2_std']

In [6]:
train = pd.read_feather('preprocessing/train')
test = pd.read_feather('preprocessing/test')

In [8]:
train.head()

,row_id,stock_id,time_id,total_volume_std,trade_log_return_realized_volatility,trade_order_count_mean,trade_pct_change_price_gmean,trade_seconds_in_bucket_count_unique,trade_size_sum,total_volume_std_150,trade_log_return_realized_volatility_150,trade_order_count_mean_150,trade_pct_change_price_gmean_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,total_volume_std_300,trade_log_return_realized_volatility_300,trade_order_count_mean_300,trade_pct_change_price_gmean_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,total_volume_std_450,trade_log_return_realized_volatility_450,trade_order_count_mean_450,trade_pct_change_price_gmean_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,target
0,0-5,0,5,138.101214,0.002006,2.750000,NaN,40.0,3179.0,142.761068,0.001701,2.433333,NaN,30.0,2069.0,136.527199,0.001308,2.571429,NaN,21.0,1587.0,116.940077,0.001060,2.642857,NaN,14.0,1042.0,0.004136
1,0-11,0,11,172.263581,0.000901,1.900000,NaN,30.0,1289.0,178.652395,0.000813,2.041667,NaN,24.0,1173.0,168.586713,0.000587,2.250000,NaN,16.0,900.0,177.264272,0.000501,2.200000,NaN,10.0,828.0,0.001445
2,0-16,0,16,138.433034,0.001961,2.720000,NaN,25.0,2161.0,135.376048,0.001621,2.950000,NaN,20.0,2010.0,120.920736,0.001137,3.166667,NaN,12.0,1189.0,116.212559,0.001048,3.666667,NaN,9.0,1085.0,0.002168
3,0-31,0,31,156.120334,0.001561,3.933333,NaN,15.0,1962.0,156.628404,0.001401,4.545455,NaN,11.0,1631.0,146.485459,0.001089,5.111111,NaN,9.0,1556.0,153.413704,0.000802,3.666667,NaN,3.0,514.0,0.002195
4,0-62,0,62,158.054066,0.000871,4.045455,NaN,22.0,1791.0,162.610706,0.000550,4.500000,NaN,16.0,1570.0,165.851509,0.000453,4.909091,NaN,11.0,1219.0,123.180227,0.000360,3.500000,NaN,4.0,43.0,0.001747


In [9]:
test.head()

,row_id,stock_id,time_id,total_volume_std,trade_log_return_realized_volatility,trade_order_count_mean,trade_pct_change_price_gmean,trade_seconds_in_bucket_count_unique,trade_size_sum,total_volume_std_150,trade_log_return_realized_volatility_150,trade_order_count_mean_150,trade_pct_change_price_gmean_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,total_volume_std_300,trade_log_return_realized_volatility_300,trade_order_count_mean_300,trade_pct_change_price_gmean_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,total_volume_std_450,trade_log_return_realized_volatility_450,trade_order_count_mean_450,trade_pct_change_price_gmean_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450
0,0-4,0,4,65.271229,0.000295,3.666667,NaN,3.0,201.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-32,0,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-34,0,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
from spica.VIF import vif_search

In [12]:
copy_train = train.copy()
copy_train.dropna(inplace=True)
copy_train = copy_train.drop(['row_id','stock_id','time_id'], axis=1)
copy_train.reset_index(drop=True, inplace=True)

In [14]:
vif_train = vif_search(copy_train)
vif_train

100%|██████████| 25/25 [00:00<00:00, 372.36it/s]


,VIF Factor,features
0,51.961673,total_volume_std
1,32.082837,trade_log_return_realized_volatility
2,22.538641,trade_order_count_mean
3,81.603267,trade_pct_change_price_gmean
4,44.496276,trade_seconds_in_bucket_count_unique
5,30.291669,trade_size_sum
6,95.270859,total_volume_std_150
7,39.984087,trade_log_return_realized_volatility_150
8,29.061777,trade_order_count_mean_150
9,175.915868,trade_pct_change_price_gmean_150


In [15]:
test_predictions = train_and_evaluate(train, test)

Training fold 1
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000531757	training's RMSPE: 0.246287	valid_1's rmse: 0.000576084	valid_1's RMSPE: 0.265825
[100]	training's rmse: 0.000518044	training's RMSPE: 0.239936	valid_1's rmse: 0.000574289	valid_1's RMSPE: 0.264997
[150]	training's rmse: 0.000508659	training's RMSPE: 0.235589	valid_1's rmse: 0.000574187	valid_1's RMSPE: 0.26495
Early stopping, best iteration is:
[128]	training's rmse: 0.000512564	training's RMSPE: 0.237397	valid_1's rmse: 0.000573724	valid_1's RMSPE: 0.264736
Training fold 2
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000532589	training's RMSPE: 0.246358	valid_1's rmse: 0.000554773	valid_1's RMSPE: 0.2573
[100]	training's rmse: 0.000518613	training's RMSPE: 0.239894	valid_1's rmse: 0.000554885	valid_1's RMSPE: 0.257352
Early stopping, best iteration is:
[88]	training's rmse: 0.00052133	training's RMSPE: 0.24115	valid_1's rmse: 0.00055336

In [18]:
copy_test = test.copy()
copy_test = copy_test.drop(['row_id','stock_id','time_id'], axis=1)

In [19]:
from spica.VIF import vif_decrese_method

In [20]:
vif_train

,VIF Factor,features
0,51.961673,total_volume_std
1,32.082837,trade_log_return_realized_volatility
2,22.538641,trade_order_count_mean
3,81.603267,trade_pct_change_price_gmean
4,44.496276,trade_seconds_in_bucket_count_unique
5,30.291669,trade_size_sum
6,95.270859,total_volume_std_150
7,39.984087,trade_log_return_realized_volatility_150
8,29.061777,trade_order_count_mean_150
9,175.915868,trade_pct_change_price_gmean_150


In [21]:
drop_train = vif_decrese_method(copy_train, 10)

100%|██████████| 25/25 [00:00<00:00, 218.47it/s]
25it [00:00, 5107.28it/s]


In [25]:
drop_train.columns

Index(['total_volume_std_450', 'trade_log_return_realized_volatility_450',
       'trade_order_count_mean_450',
       'trade_seconds_in_bucket_count_unique_450', 'trade_size_sum_450',
       'target'],
      dtype='object')

In [29]:
from tqdm import tqdm
drop_list = []
for columns in tqdm(train.columns):
    if columns not in drop_train.columns:
        drop_list.append(columns)

drop_list.remove('row_id')
drop_list.remove('stock_id')
drop_list.remove('time_id')
drop_list

100%|██████████| 28/28 [00:00<00:00, 57966.69it/s]


['total_volume_std',
 'trade_log_return_realized_volatility',
 'trade_order_count_mean',
 'trade_pct_change_price_gmean',
 'trade_seconds_in_bucket_count_unique',
 'trade_size_sum',
 'total_volume_std_150',
 'trade_log_return_realized_volatility_150',
 'trade_order_count_mean_150',
 'trade_pct_change_price_gmean_150',
 'trade_seconds_in_bucket_count_unique_150',
 'trade_size_sum_150',
 'total_volume_std_300',
 'trade_log_return_realized_volatility_300',
 'trade_order_count_mean_300',
 'trade_pct_change_price_gmean_300',
 'trade_seconds_in_bucket_count_unique_300',
 'trade_size_sum_300',
 'trade_pct_change_price_gmean_450']

In [30]:
train = train.drop(drop_list, axis=1)
test = test.drop(drop_list, axis=1)

In [32]:
test_predictions = train_and_evaluate(train, test)

Training fold 1
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000620574	training's RMSPE: 0.287423	valid_1's rmse: 0.000653726	valid_1's RMSPE: 0.301652
[100]	training's rmse: 0.000610235	training's RMSPE: 0.282635	valid_1's rmse: 0.000655537	valid_1's RMSPE: 0.302487
Early stopping, best iteration is:
[54]	training's rmse: 0.000619442	training's RMSPE: 0.286899	valid_1's rmse: 0.000653498	valid_1's RMSPE: 0.301546
Training fold 2
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000621893	training's RMSPE: 0.287667	valid_1's rmse: 0.000641732	valid_1's RMSPE: 0.297631
Early stopping, best iteration is:
[46]	training's rmse: 0.000623104	training's RMSPE: 0.288228	valid_1's rmse: 0.000641369	valid_1's RMSPE: 0.297462
Training fold 3
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.000620778	training's RMSPE: 0.287327	valid_1's rmse: 0.000648874	valid_1's RMSPE: 0.30021
[100]	tra